# Spectra Intelligence Dynamic Analysis API-s
This notebook demonstrates how to use the Dynamic Analysis API-s, namely the Dynamic Analysis and Dynamic Analysis Report. The first one mentioned here handles submitting a cloud sample or a URL to dynamic analysis and the second one handles returning a comprehensive report.
**NOTE:** If pasted into a Python file in the displayed order, all code cells can also work as a Python script.

### Covered API classes
This notebook covers examples for the following API class:
- **DynamicAnalysis** (*TCA-0207 and TCA-0106*)
    - The ReversingLabs SDK couples both API-s under one Python class.

### Credentials
Credentials are loaded from a local file instead of being written here in plain text.
To learn how to creat the credentials file, see the **Storing and using the credentials** section in the [README file](./README.md)

### 1. Importing the required classes
First, we will import the required API class from the ticloud module.

In [ ]:
from ReversingLabs.SDK.ticloud import DynamicAnalysis

### 2. Loading the credentials
Next, we will load our Spectra Intelligence credentials from the local `ticloud_credentials.json` file.
**NOTE: Instead of doing this step, you can paste your credentials while creating the Python object in the following step.**

In [ ]:
import json


CREDENTIALS = json.load(open("ticloud_credentials.json"))
USERNAME = CREDENTIALS.get("username")
PASSWORD = CREDENTIALS.get("password")
USER_AGENT = json.load(open('../user_agent.json'))["user_agent"]

### 3. Creating the DynamicAnalysis object
After we create an object from the DynamicAnalysis class, we will be able to use its methods.

In [ ]:
dynamic_analysis = DynamicAnalysis(
    host="https://data.reversinglabs.com",
    username=USERNAME,
    password=PASSWORD,
    user_agent=USER_AGENT
)

### 4. Submitting a sample for Dynamic Analysis
To submit a sample for dynamic analysis, we need the sample's SHA1 hash instead of the sample's content. That is because the sample needs to already be present in Spectra Intelligence to be available for detonation.  
To see how to upload a sample to Spectra Intelligence, see section 6. or the [Sample management notebook](./sample_management.ipynb).  
Please change the `example_sha1_hash` into an actual SHA1 hash of a sample found in the cloud.

In [ ]:
detonate1 = dynamic_analysis.detonate_sample(
    sample_sha1="example_sha1_hash",
    platform="windows10"
)

print(detonate1.text)

If the detonation request was successful, we will see a response message stating that the analysis has been requested.  
**NOTE:** After requesting Dynamic Analysis on a sample, it may take some time for it to be finished and the results ready. You can periodically try to fetch the results to see the status.

### 5. Getting the Dynamic Analysis results for a sample
After the Dynamic Analysis is finished, we can move on to getting the results for our sample.

In [ ]:
results = dynamic_analysis.get_dynamic_analysis_results(sample_hash="example_sha1_hash")

print(results.text)

This is the simplest request we can make to acquire the results for our analyzed sample. There are other options available:

#### A specific analysis from the past

In [ ]:
results = dynamic_analysis.get_dynamic_analysis_results(
    sample_hash="example_sha1_hash",
    analysis_id="specific_analysis_id"
)

print(results.text)

When returning the analysis results in general, each returned analysis has its own ID that can be used as this argument.

#### The latest analysis

In [ ]:
results = dynamic_analysis.get_dynamic_analysis_results(
    sample_hash="example_sha1_hash",
    latest=True
)

print(results.text)

This request will return the results of only the latest analysis that we performed.

### 6. Uploading the sample to the cloud
For a sample to be available for dynamic analysis it **needs to be present on Spectra Intelligence**. If a sample is not present in Spectra Intelligence, we first need to upload it using the **FileUpload** class.

In [ ]:
from ReversingLabs.SDK.ticloud import FileUpload


upload = FileUpload(
    host="https://data.reversinglabs.com",
    username=USERNAME,
    password=PASSWORD,
    user_agent=USER_AGENT
)

_ = upload.upload_sample_from_path(file_path="sample.exe")

After the sample is uploaded to the cloud, we can request Dynamic Analysis on it. Again, to submit the sample for Dynamic Analysis, we need to use its SHA1 hash.

In [ ]:
detonate2 = dynamic_analysis.detonate_sample(
    sample_sha1="example_sha1_hash",
    platform="windows7"
)

### 7. Submitting a URL for Dynamic Analysis
Apart from submitting only samples for Dynamic Analysis, we can also do the same for URL-s.  
The procedure is, for the most part, the same, with the0 difference being in using a different "detonate" method and a different method parameter while fetching the results. To see how it differs from detonation and result fetching for samples, see the examples below.

**Note:** It is highly recommended **not to open the URL-s** used as examples in this cookbook as many of them may be classified as containing malicious software and/or indicators and can be **harmful for you computer**. For this example, we used a placeholder URL.

In [ ]:
detonate3 = dynamic_analysis.detonate_url(
    url_string="http://some.example.url./aaa",
    platform="windows10"
)

print(detonate3.text)

The response text that we received tells us the following info:
- The analysis started.
- We got the SHA-1, the Base64 and the actual string of our submitted URL.
- We also got the ID of the started analysis.

It is useful to save those values since we will be using them to access the URL Dynamic Analysis report.

### 8. Getting the Dynamic Analysis results for a URL
To fetch the Dynamic Analysis results for our submitted URL, we can use one of the following as an input parameter:
- The SHA-1 of the URL.
- The URL string itself.

For demonstration purposes, we will show you how to use both - in the order written above.

In [ ]:
# Example 1
results_sha1 = dynamic_analysis.get_dynamic_analysis_results(
    url_sha1="27c5d0351a92e7be52884c6aa863d25665cb6b32",
    analysis_id="6c8d9c60-2d9c-4c27-bc15-150b7e364d6c"
)

# Example 2
results_url = dynamic_analysis.get_dynamic_analysis_results(
    url="http://some.example.url./aaa",
    latest=True
)

print(results_sha1.text)
print(results_url.text)

Keep in mind that these are all placeholder values that serve only as examples of use. We did not use actual malicious URL-s for security purposes.  
What did we do in these two examples?

Example 1:
- We used the URL-s SHA-1 as input.
- We used the analysis ID to define which analysis we need.

Example 2:
- We used the actual URL string as input.
- We stated that we want the latest analysis returned.

Apart from defining the desired analysis using the `analysis_id` and `latest` parameters, we can also omit them completely and receive a merged report from all the analyses for this URL so far. However, keep in mind that, if the report is not yet ready, you will receive `404` as a status code.